# NLP - Lab 04
## Adrien Giget, Tanguy Malandain, Denis Stojiljkovic

## Introduction to Natural Language Processing 01

Nous allons utiliser FastText pour implémenter classificateur de sentiments.

# FastText (8 points)

## Imports

In [1]:
import fasttext
from typing import Dict, Any
import re

import os
from sklearn.model_selection import train_test_split

from datasets import load_dataset
import numpy as np

In [2]:
# instantiate a Pseudo-random number generator (PRNG)
rng = np.random.default_rng(420)

## Get data

In [3]:
dataset = load_dataset("imdb")
del dataset["unsupervised"]

Found cached dataset imdb (/home/adrien/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

### (2 points) Turn the dataset into a dataset compatible with Fastext (see the Tips on using FastText section a bit lower).
For pretreatment, only apply lower casing and punctuation removal.


We re-use the code from the previous lab for pretreatment.

In [4]:
def get_lower(comment: Dict[str, Any]) -> Dict[str, Any]:
    """
    :param comment: Actual costumer comment as a dict : { str : Any }
    :return: a modified dict : { str : Any }
    """
    comment["text"] = comment["text"].lower()
    return comment

def remove_angle_brackets(comment: Dict[str, Any]) -> Dict[str, Any]:
    """
    Remove everything inside < > characters (usually <br > from scraping)
    :param comment: Actual costumer comment as a dict : { str : Any }
    :return: a modified dict : { str : Any }
    """
    comment["text"] = re.sub(f'<.*?>', '', comment["text"])
    return comment

def remove_unknown_unicode(comment: Dict[str, Any]) -> Dict[str, Any]:
    """
    Remove all non-usual unicode characters (as —)
    :param comment: Actual costumer comment as a dict : { str : Any }
    :return: a modified dict : { str : Any }
    """
    comment["text"] = re.sub(r"(\\x\S+)", '', repr(comment["text"])[1:-1])
    return comment

def remove_backslash(comment: Dict[str, Any]) -> Dict[str, Any]:
    """
    Remove all backslash (usually as "it\'s")
    :param comment: Actual costumer comment as a dict : { str : Any }
    :return: a modified dict : { str : Any }
    """
    comment["text"] = re.sub(r"\\", '', comment["text"])
    return comment

def remove_ponctuation(comment: Dict[str, Any]) -> Dict[str, Any]:
    """
    Remove non-wanted ponctuations as -_\(\)\".,`:;* that don't add much syntactical meaning.
    :param comment: Actual costumer comment as a dict : { str : Any }
    :return: a modified dict : { str : Any }
    """
    comment["text"] = re.sub(r"[-_\(\)\".,`:;*]", ' ', comment["text"])
    comment["text"] = re.sub(r"[(!?)]", r" \g<0> ", comment["text"]) # We want to keep ? and ! in the split
    return comment

def remove_all(comment: Dict[str, Any]) -> Dict[str, Any]:
    """
    Apply all above
    :param comment: Actual costumer comment as a dict : { str : Any }
    :return: a modified dict : { str : Any }
    """
    return remove_ponctuation(remove_backslash(remove_unknown_unicode(remove_angle_brackets(get_lower(comment)))))

In [5]:
adapted_data = dataset.map(remove_all)

Loading cached processed dataset at /home/adrien/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-608093fc9e53df62.arrow
Loading cached processed dataset at /home/adrien/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-76ab34fc70dca627.arrow


## Création des fichiers nécessaires à FastText

In [6]:
# Charger les données
train_data = adapted_data["train"]
test_data = adapted_data["test"]

# Diviser les données d'entraînement en données d'entraînement et de validation (20% pour la validation)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [7]:
# Créer un dossier 'data' s'il n'existe pas
os.makedirs("data", exist_ok=True)

In [8]:
# Fonction pour écrire les données dans un fichier FastText
def write_fasttext_data(data_text, data_label, filename):
    with open(filename, "w+", encoding="utf-8") as f:
        for index in range(len(data_text)):
            label = data_label[index]
            text = data_text[index]
            f.write(f"__label__{label} {text}\n")


In [9]:
# Écrire les données d'entraînement, de validation et de test dans des fichiers séparés
write_fasttext_data(train_data["text"], train_data["label"], "data/train.txt")
write_fasttext_data(val_data["text"], val_data["label"], "data/validation.txt")
write_fasttext_data(test_data["text"], test_data["label"], "data/test.txt")

### (2 points) Train a FastText classifier with default parameters on the training data, and evaluate it on the test data using accuracy.

In [10]:
%%time

# Entraîner un classificateur FastText avec les paramètres par défaut
model = fasttext.train_supervised("data/train.txt")

Read 4M words
Number of words:  83253
Number of labels: 2
Progress: 100.0% words/sec/thread: 3932869 lr: -0.000020 avg.loss:  0.458177 ETA:   0h 0m 0s

CPU times: user 4.93 s, sys: 122 ms, total: 5.05 s
Wall time: 721 ms


Progress: 100.0% words/sec/thread: 3930361 lr:  0.000000 avg.loss:  0.458177 ETA:   0h 0m 0s


In [11]:
# Prédire les étiquettes pour les données de test
test_predictions = [model.predict(text)[0][0] for text in test_data["text"]]

# Calculer l'accuracy
correct_predictions = sum(1 for true_label, predicted_label in zip(test_data["label"], test_predictions) if f'__label__{true_label}' == predicted_label)
accuracy = correct_predictions / len(test_data["label"])

print(f"Accuracy: {accuracy:.2%}")

Accuracy: 86.80%


Malgré que le modèle est appris de façon non supervise, il obtient une très bonne accuracy (bien plus que le modèle du dernier lab).
Cela est sans doute dû au fait que FastText capture plus efficace les relations entre les mots que nos modèles probabilistes.

### (2 points) Use the hyperparameters search functionality of FastText and repeat step 2.

* To do so, you'll need to split your training set into a training and a validation set.
* Let the model search for 5 minutes (it's the default search time).
* Don't forget to shuffle (and stratify) your splits. The dataset has its entry ordered by label (0s first, then 1s). Feeding the classifier one class and then the second can mess with its performances.


In [12]:
%%time

# Entraîner un classificateur FastText avec recherche d'hyperparamètres (cela se fait automatiquement quand on utilise train_supervised())
model_2 = fasttext.train_supervised("data/train.txt", autotuneValidationFile="data/validation.txt", autotuneDuration=300)

Progress: 100.0% Trials:   11 Best score:  0.900800 ETA:   0h 0m 0s
Training again with best arguments
Read 4M words
Number of words:  83253
Number of labels: 2
Progress:  98.8% words/sec/thread: 1629395 lr:  0.001047 avg.loss:  0.045944 ETA:   0h 0m 0s

CPU times: user 1h 16min 17s, sys: 15.4 s, total: 1h 16min 33s
Wall time: 5min 20s


Progress: 100.0% words/sec/thread: 1624233 lr:  0.000000 avg.loss:  0.045442 ETA:   0h 0m 0s


In [13]:
# Prédire les étiquettes pour les données de test
test_predictions_2 = [model_2.predict(text)[0][0] for text in test_data["text"]]

# Calculer l'accuracy
correct_predictions_2 = sum(1 for true_label, predicted_label in zip(test_data["label"], test_predictions_2) if f'__label__{true_label}' == predicted_label)
accuracy_2 = correct_predictions_2 / len(test_data["label"])

print(f"Accuracy: {accuracy_2:.2%}")

Accuracy: 89.49%


On remarque que l'accuracy est légèrement meilleure avec la recherche d'hyperparamètres.

### (1 points) Look at the differences between the default model and the attributes found with hyperparameters search. How do the two models differ?

* Only refer to the attributes you think are interesting.
* See the Tips on using FastText (just below) for help.


In [14]:
# Afficher les attributs intéressants pour les deux modèles
print("Modèle par défaut :")
print(f"  Taux d'apprentissage (lr) : {model.lr}")
print(f"  Taille du vecteur (dim) : {model.dim}")
print(f"  Nombre d'époques (epoch) : {model.epoch}")

print("\nModèle avec recherche d'hyperparamètres :")
print(f"  Taux d'apprentissage (lr) : {model_2.lr}")
print(f"  Taille du vecteur (dim) : {model_2.dim}")
print(f"  Nombre d'époques (epoch) : {model_2.epoch}")

Modèle par défaut :
  Taux d'apprentissage (lr) : 0.1
  Taille du vecteur (dim) : 100
  Nombre d'époques (epoch) : 5

Modèle avec recherche d'hyperparamètres :
  Taux d'apprentissage (lr) : 0.08499425639667486
  Taille du vecteur (dim) : 92
  Nombre d'époques (epoch) : 100


Dans ces deux modèles, la différence d'hyperparamètres vient surtout de la différence du taux d'apprentissage et du nombre d'époques.

Mais il est important de rappeler que l'accuracy des deux modèles est très proche. Ainsi, on peut voir la version avancée seulement
comme une version ayant apprise plus lentement et plus longtemps que la version par défaut, ce qui permet un fit légèrement plus précis.

### (Bonus point) Why is it likely that the attributes minn and maxn are at 0 after an hyperparameter search on our data?

* Hint: on what language are we working?


Dans le contexte de nos données, il est possible que les valeurs minn et maxn soient fixées à 0 après la recherche d'hyperparamètres, car nous traitons des commentaires rédigés en anglais. Les sous-mots (n-grams) pourraient ne pas être aussi pertinents que pour d'autres langues, telles que les langues agglutinantes. FastText cherche à déterminer les meilleurs hyperparamètres afin de réduire l'erreur de validation. Si les n-grams n'améliorent pas significativement les performances du modèle, la recherche d'hyperparamètres pourrait décider de ne pas les inclure, d'où des valeurs minn et maxn égales à 0.